## Evaluation

Evaluate model on sample evaluation data.

In [2]:
!pip install statsmodels

  Using cached https://files.pythonhosted.org/packages/88/9a/fb4ec96ff528fa6426063b1357b4aad9af4cb7dafc53ac1d5a5762ffeb91/statsmodels-0.12.1-cp38-cp38-macosx_10_13_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ea/0c/5f61f1a3d4385d6bf83b83ea495068857ff8dfb89e74824c6e9eb63286d8/patsy-0.5.1-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [47]:
import numpy as np
import pandas as pd
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from sklearn.metrics import mean_squared_error
import pickle
warnings.filterwarnings('ignore')

### Copy Data Locally

In [18]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [19]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f9d3ac29400>)

In [20]:
# Load historical data from local file
oxford_data = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [21]:
ID_COLS = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
CASES_COL = ['NewCases']
NPI_COLS = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
NB_LOOKBACK_DAYS = 30
# For testing, restrict training data to that before a hypothetical predictor submission date
SUBMISSION_DATE = np.datetime64('2020-12-20')

In [23]:
# Add GeoID column that combines CountryName and RegionName for easier manipulation of data",
oxford_data['GeoID'] = oxford_data['CountryName'] + '__' + oxford_data['RegionName'].astype(str)

# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in NPI_COLS:
    oxford_data.update(oxford_data.groupby(['CountryName', 'RegionName'])[npi_col].ffill().fillna(0))

# Add new cases column
oxford_data['NewCases'] = oxford_data.groupby('GeoID').ConfirmedCases.diff().fillna(0)
# Fill any missing case values by interpolation and setting NaNs to 0
oxford_data.update(oxford_data.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))
# Keep only the id and cases columns
oxford_data = oxford_data[ID_COLS + CASES_COL + NPI_COLS]

In [29]:
oxford_data = oxford_data[oxford_data.Date <= SUBMISSION_DATE]

In [37]:
oxford_data.head()

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
arima_regions = [
  'Andorra',
  'Aruba',
  'Bahamas',
  'Barbados',
  'Belize',
  'Bermuda',
  'Bhutan',
  'Brunei',
  'Cape Verde',
  'Central African Republic',
  'Comoros',
  'Congo',
  'Cyprus',
  'Djibouti',
  'Dominica',
  'Faeroe Islands',
  'Fiji',
  'Gambia',
  'Greenland',
  'Guam',
  'Iceland',
  'Jamaica',
  'Laos',
  'Macao',
  'Mauritius',
  'Monaco',
  'Mongolia',
  'New Zealand',
  'Norway',
  'Qatar',
  'San Marino',
  'Seychelles',
  'Solomon Islands',
  'South Sudan',
  'Tanzania',
  'Timor-Leste',
  'Togo',
  'Trinidad and Tobago',
  'Vanuatu',
]

In [39]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [53]:
for g in arima_regions:
    print('\nPredicting for', g)

    # Pull out all relevant data for country c
    hist_cases_gdf = oxford_data[oxford_data.CountryName == g]
    past_cases = np.array(hist_cases_gdf[CASES_COL])

    # ~TODO~
    # step 1: pre-process hist_cases_gdf into history array for SARIMAX model
    timesteps = hist_cases_gdf['NewCases'].values.tolist()
    timesteps = moving_average(timesteps, 7)
    data = pd.DataFrame(data=timesteps, columns=['new_cases']).squeeze()
    X = data.values

    # step 2: copy from arima_model.py how to fit SARIMAX model with history & perform grid search for hyperparameters 
    best_score, best_cfg, best_pred = float("inf"), None, None
    for p in range(0, 3):
        for d in range(0, 3):
            for q in range(0, 3):
                train, test = X[:-7], X[-7:]
                sarimax_order = (p,d,q)
                # print('trying:', sarimax_order)
                pred = list()
                for t in range(len(test)): # train using rolling window
                    model = SARIMAX(train, order=sarimax_order, initialization='approximate_diffuse')
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    yhat = output[0]
                    yhat = max(0, yhat)  # Do not allow predicting negative cases
                    pred.append(yhat)
                    train = np.append(train, yhat)
                error = sqrt(mean_squared_error(test, pred)) # RMSE
                if error < best_score:
                    best_score, best_cfg, best_pred = error, sarimax_order, pred
    with open(f'models/{g}.pkl', 'wb') as f:
      pickle.dump(best_cfg, f)


Predicting for Andorra

Predicting for Aruba

Predicting for Bahamas

Predicting for Barbados

Predicting for Belize

Predicting for Bermuda

Predicting for Bhutan

Predicting for Brunei

Predicting for Cape Verde

Predicting for Central African Republic

Predicting for Comoros

Predicting for Congo

Predicting for Cyprus

Predicting for Djibouti

Predicting for Dominica

Predicting for Faeroe Islands

Predicting for Fiji

Predicting for Gambia

Predicting for Greenland

Predicting for Guam

Predicting for Iceland

Predicting for Jamaica

Predicting for Laos

Predicting for Macao

Predicting for Mauritius

Predicting for Monaco

Predicting for Mongolia

Predicting for New Zealand

Predicting for Norway

Predicting for Qatar

Predicting for San Marino

Predicting for Seychelles

Predicting for Solomon Islands

Predicting for South Sudan

Predicting for Tanzania

Predicting for Timor-Leste

Predicting for Togo

Predicting for Trinidad and Tobago

Predicting for Vanuatu


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
%%time
!python predict.py -s 2020-12-01 -e 2020-12-15 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-12-01_2020-12-15.csv

In [39]:
!head predictions/2020-12-01_2020-12-15.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,nan,2020-12-01,14.952073871634664
Aruba,nan,2020-12-02,14.307590223934163
Aruba,nan,2020-12-03,13.808117290464779
Aruba,nan,2020-12-04,13.240991753033573
Aruba,nan,2020-12-05,12.705430399430888
Aruba,nan,2020-12-06,12.155141510272477
Aruba,nan,2020-12-07,11.611724743334479
Aruba,nan,2020-12-08,11.065101139604401
Aruba,nan,2020-12-09,10.51997407415748


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [63]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict-new.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [61]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-29
End date: 2021-06-27


In [62]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=arima_regions, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


In [64]:
scenario_df.CountryName.unique()

array(['Andorra', 'Aruba', 'Bahamas', 'Barbados', 'Belize', 'Bermuda',
       'Bhutan', 'Brunei', 'Cape Verde', 'Central African Republic',
       'Comoros', 'Congo', 'Cyprus', 'Djibouti', 'Dominica',
       'Faeroe Islands', 'Fiji', 'Gambia', 'Greenland', 'Guam', 'Iceland',
       'Jamaica', 'Laos', 'Macao', 'Mauritius', 'Monaco', 'Mongolia',
       'New Zealand', 'Norway', 'Qatar', 'San Marino', 'Seychelles',
       'Solomon Islands', 'South Sudan', 'Tanzania', 'Timor-Leste',
       'Togo', 'Trinidad and Tobago', 'Vanuatu'], dtype=object)

### Check it

In [69]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-29 to 2021-06-27...
HERE

Predicting for Aruba__nan
2020-12-23: 12.0 - Skipped (intermediate missing daily cases)
2020-12-24: 12.0 - Skipped (intermediate missing daily cases)
2020-12-25: 12.0 - Skipped (intermediate missing daily cases)
2020-12-26: 12.0 - Skipped (intermediate missing daily cases)
2020-12-27: 12.0 - Skipped (intermediate missing daily cases)
2020-12-28: 12.0 - Skipped (intermediate missing daily cases)
2020-12-29: 12.0
2020-12-30: 12.0
2020-12-31: 12.0
2021-01-01: 12.0
2021-01-02: 12.0
2021-01-03: 12.0
2021-01-04: 12.0
2021-01-05: 12.0
2021-01-06: 12.0
2021-01-07: 12.0
2021-01-08: 12.0
2021-01-09: 12.0
2021-01-10: 12.0
2021-01-11: 12.0
2021-01-12: 12.0
2021-01-13: 12.0
2021-01-14: 12.0
2021-01-15: 12.0
2021-01-16: 12.0
2021-01-17: 12.0
2021-01-18: 12.0
2021-01-19: 12.0
2021-01-20: 12.0
2021-01-21: 12.0
2021-01-22: 12.0
2021-01-23: 12.0
2021-01-24: 12.0
2021-01-25: 12.0
2021-01-26: 12.0
2021-01-27: 12.0
2021-01-28: 12.0
2021-01-29: 1

2021-05-12: 9.988594092754006
2021-05-13: 9.927989689653558
2021-05-14: 9.86777298415502
2021-05-15: 9.807910593937516
2021-05-16: 9.748418979761752
2021-05-17: 9.689270068946009
2021-05-18: 9.630487969558343
2021-05-19: 9.57203095023239
2021-05-20: 9.513957227141844
2021-05-21: 9.456222711300295
2021-05-22: 9.398837773584596
2021-05-23: 9.341809370339272
2021-05-24: 9.285114756191064
2021-05-25: 9.228765138155023
2021-05-26: 9.172755548697213
2021-05-27: 9.117079263396272
2021-05-28: 9.061742465408706
2021-05-29: 9.006742326313958
2021-05-30: 8.952074962517177
2021-05-31: 8.897698066050939
2021-06-01: 8.843699499116784
2021-06-02: 8.790014108313226
2021-06-03: 8.736650997842412
2021-06-04: 8.683617600655017
2021-06-05: 8.630916106519182
2021-06-06: 8.578519515907711
2021-06-07: 8.52644493555572
2021-06-08: 8.47468550431974
2021-06-09: 8.423233513880017
2021-06-10: 8.372104860525656
2021-06-11: 8.321272520312732
2021-06-12: 8.27076323109061
2021-06-13: 8.220545233376262
2021-06-14: 8.1

2020-12-30: 6.661338147750971e-16
2020-12-31: 8.881784197001296e-16
2021-01-01: 1.110223024625162e-15
2021-01-02: 1.3322676295501946e-15
2021-01-03: 1.554312234475227e-15
2021-01-04: 1.7763568394002595e-15
2021-01-05: 1.998401444325292e-15
2021-01-06: 2.2204460492503245e-15
2021-01-07: 2.442490654175357e-15
2021-01-08: 2.6645352591003895e-15
2021-01-09: 2.886579864025422e-15
2021-01-10: 3.1086244689504545e-15
2021-01-11: 3.330669073875487e-15
2021-01-12: 3.55271367880052e-15
2021-01-13: 3.774758283725553e-15
2021-01-14: 3.996802888650586e-15
2021-01-15: 4.2188474935756185e-15
2021-01-16: 4.4408920985006514e-15
2021-01-17: 4.662936703425684e-15
2021-01-18: 4.884981308350717e-15
2021-01-19: 5.10702591327575e-15
2021-01-20: 5.329070518200783e-15
2021-01-21: 5.551115123125816e-15
2021-01-22: 5.773159728050849e-15
2021-01-23: 5.995204332975882e-15
2021-01-24: 6.2172489379009145e-15
2021-01-25: 6.439293542825947e-15
2021-01-26: 6.66133814775098e-15
2021-01-27: 6.883382752676013e-15
2021-01-2

2021-02-17: 11.779015075589827
2021-02-18: 11.808016864915297
2021-02-19: 11.836998238229567
2021-02-20: 11.86596280400218
2021-02-21: 11.894926100285032
2021-02-22: 11.923889517349227
2021-02-23: 11.952852839919403
2021-02-24: 11.981816566394649
2021-02-25: 12.010781157206816
2021-02-26: 12.039746064690183
2021-02-27: 12.068714255903858
2021-02-28: 12.097678531851797
2021-03-01: 12.126664530362898
2021-03-02: 12.155633605133543
2021-03-03: 12.18460851265339
2021-03-04: 12.213576514487775
2021-03-05: 12.242538775095856
2021-03-06: 12.271557763478816
2021-03-07: 12.300535645520146
2021-03-08: 12.32949432038948
2021-03-09: 12.358459878540959
2021-03-10: 12.387423077643252
2021-03-11: 12.416386936218805
2021-03-12: 12.445349990585408
2021-03-13: 12.474313547054136
2021-03-14: 12.503277388588499
2021-03-15: 12.532240813224865
2021-03-16: 12.56120729462268
2021-03-17: 12.590171661393304
2021-03-18: 12.619139738600659
2021-03-19: 12.648105228100455
2021-03-20: 12.67707130691917
2021-03-21: 1

2021-03-27: 1.8167752077727175
2021-03-28: 1.820864035803265
2021-03-29: 1.8249528754816213
2021-03-30: 1.8290416917413708
2021-03-31: 1.8331305427163957
2021-04-01: 1.8372193938807064
2021-04-02: 1.8413082455779448
2021-04-03: 1.8453971014436876
2021-04-04: 1.8494859486188955
2021-04-05: 1.8535747933390432
2021-04-06: 1.857663652271034
2021-04-07: 1.8617524987343252
2021-04-08: 1.8658413593935326
2021-04-09: 1.869930225325244
2021-04-10: 1.8740190872330877
2021-04-11: 1.8781079519908324
2021-04-12: 1.8821960465959067
2021-04-13: 1.8862848375126098
2021-04-14: 1.8903736895212848
2021-04-15: 1.8944625172468428
2021-04-16: 1.8985513810071102
2021-04-17: 1.9026402343032698
2021-04-18: 1.906729101271857
2021-04-19: 1.9108179747722696
2021-04-20: 1.9149068474944508
2021-04-21: 1.9189957155180468
2021-04-22: 1.9230845889141088
2021-04-23: 1.9271734099837308
2021-04-24: 1.931262281702731
2021-04-25: 1.9353511512060912
2021-04-26: 1.939440025996291
2021-04-27: 1.9435288980384142
2021-04-28: 1.

...

2021-01-06: 0.8916237725314801
2021-01-07: 0.8936451593925795
2021-01-08: 0.8956488081294557
2021-01-09: 0.8979496945383892
2021-01-10: 0.9002537179681559
2021-01-11: 0.9025685013087135
2021-01-12: 0.9048779829491633
2021-01-13: 0.9071799206331879
2021-01-14: 0.9093827697784567
2021-01-15: 0.9115789656569342
2021-01-16: 0.9137704049987344
2021-01-17: 0.9159592622040869
2021-01-18: 0.9172663696837635
2021-01-19: 0.9185712125715721
2021-01-20: 0.9198759316361286
2021-01-21: 0.9221693928428896
2021-01-22: 0.9239916158497639
2021-01-23: 0.925774100879381
2021-01-24: 0.9278034474593836
2021-01-25: 0.9298309815696959
2021-01-26: 0.9318566428428494
2021-01-27: 0.9338808482019294
2021-01-28: 0.9344747340377083
2021-01-29: 0.935069323616458
2021-01-30: 0.9356649378700541
2021-01-31: 0.9362615783897361
2021-02-01: 0.9368593873517166
2021-02-02: 0.9374584618080903
2021-02-03: 0.938058778676674
2021-02-04: 0.9386604756268486
2021-02-05: 0.9392634844100656
2021-02-06: 0.9398677185047393
2021-02-07:

2021-03-12: 0.7120995262271098
2021-03-13: 0.7117897618274367
2021-03-14: 0.7114961894133238
2021-03-15: 0.7111977566251866
2021-03-16: 0.7109002672773317
2021-03-17: 0.7106014824677385
2021-03-18: 0.7103059043296913
2021-03-19: 0.710023248617091
2021-03-20: 0.7097329902372067
2021-03-21: 0.709433896977276
2021-03-22: 0.7091403376132658
2021-03-23: 0.70885016760694
2021-03-24: 0.7085605008681409
2021-03-25: 0.7082732592045868
2021-03-26: 0.7079962397063149
2021-03-27: 0.7077003569885489
2021-03-28: 0.7074122628915351
2021-03-29: 0.707126370406397
2021-03-30: 0.7068359214003487
2021-03-31: 0.7065467276471441
2021-04-01: 0.7062629795181019
2021-04-02: 0.7059772623989163
2021-04-03: 0.7056916191926087
2021-04-04: 0.705409001078695
2021-04-05: 0.7051280161235047
2021-04-06: 0.7048428928496447
2021-04-07: 0.7045642743326658
2021-04-08: 0.7042807378481101
2021-04-09: 0.703998608109434
2021-04-10: 0.7037153141646616
2021-04-11: 0.7034386771747627
2021-04-12: 0.7031562022266167
2021-04-13: 0.7

2021-05-20: 24.849349407313408
2021-05-21: 24.899250923826827
2021-05-22: 24.949150317275002
2021-05-23: 24.999039619873898
2021-05-24: 25.04892899630114
2021-05-25: 25.098818486753164
2021-05-26: 25.148716746416945
2021-05-27: 25.198614385209
2021-05-28: 25.24850430324335
2021-05-29: 25.298406058175914
2021-05-30: 25.348305363053367
2021-05-31: 25.398196858774874
2021-06-01: 25.448098291751638
2021-06-02: 25.49799685583034
2021-06-03: 25.547897000943838
2021-06-04: 25.597798245586123
2021-06-05: 25.647699236923234
2021-06-06: 25.697599012639984
2021-06-07: 25.74749587614395
2021-06-08: 25.79738856381912
2021-06-09: 25.84727655953156
2021-06-10: 25.897173895038247
2021-06-11: 25.947075212842577
2021-06-12: 25.99697358369087
2021-06-13: 26.046874944298658
2021-06-14: 26.09677618471784
2021-06-15: 26.146677453382004
2021-06-16: 26.196575315207383
2021-06-17: 26.24647670000811
2021-06-18: 26.296374302208225
2021-06-19: 26.34627271493357
2021-06-20: 26.39615884096636
2021-06-21: 26.4460602

2021-04-20: 13.764584549409205
2021-04-21: 13.764584549409205
2021-04-22: 13.764584549409205
2021-04-23: 13.764584549409205
2021-04-24: 13.764584549409205
2021-04-25: 13.764584549409205
2021-04-26: 13.764584549409205
2021-04-27: 13.764584549409205
2021-04-28: 13.764584549409205
2021-04-29: 13.764584549409205
2021-04-30: 13.764584549409205
2021-05-01: 13.764584549409205
2021-05-02: 13.764584549409205
2021-05-03: 13.764584549409205
2021-05-04: 13.764584549409205
2021-05-05: 13.764584549409205
2021-05-06: 13.764584549409205
2021-05-07: 13.764584549409205
2021-05-08: 13.764584549409205
2021-05-09: 13.764584549409205
2021-05-10: 13.764584549409205
2021-05-11: 13.764584549409205
2021-05-12: 13.764584549409205
2021-05-13: 13.764584549409205
2021-05-14: 13.764584549409205
2021-05-15: 13.764584549409205
2021-05-16: 13.764584549409205
2021-05-17: 13.764584549409205
2021-05-18: 13.764584549409205
2021-05-19: 13.764584549409205
2021-05-20: 13.764584549409205
2021-05-21: 13.764584549409205
2021-05-

2021-01-21: 0
2021-01-22: 0
2021-01-23: 0
2021-01-24: 0
2021-01-25: 0
2021-01-26: 0
2021-01-27: 0
2021-01-28: 0
2021-01-29: 0
2021-01-30: 0
2021-01-31: 0
2021-02-01: 0
2021-02-02: 0
2021-02-03: 0
2021-02-04: 0
2021-02-05: 0
2021-02-06: 0
2021-02-07: 0
2021-02-08: 0
2021-02-09: 0
2021-02-10: 0
2021-02-11: 0
2021-02-12: 0
2021-02-13: 0
2021-02-14: 0
2021-02-15: 0
2021-02-16: 0
2021-02-17: 0
2021-02-18: 0
2021-02-19: 0
2021-02-20: 0
2021-02-21: 0
2021-02-22: 0
2021-02-23: 0
2021-02-24: 0
2021-02-25: 0
2021-02-26: 0
2021-02-27: 0
2021-02-28: 0
2021-03-01: 0
2021-03-02: 0
2021-03-03: 0
2021-03-04: 0
2021-03-05: 0
2021-03-06: 0
2021-03-07: 0
2021-03-08: 0
2021-03-09: 0
2021-03-10: 0
2021-03-11: 0
2021-03-12: 0
2021-03-13: 0
2021-03-14: 0
2021-03-15: 0
2021-03-16: 0
2021-03-17: 0
2021-03-18: 0
2021-03-19: 0
2021-03-20: 0
2021-03-21: 0
2021-03-22: 0
2021-03-23: 0
2021-03-24: 0
2021-03-25: 0
2021-03-26: 0
2021-03-27: 0
2021-03-28: 0
2021-03-29: 0
2021-03-30: 0
2021-03-31: 0
2021-04-01: 0
2021-0